In [125]:
import requests
import html
from bs4 import BeautifulSoup
from IPython.display import display
import re
import pandas as pd

In [162]:
def cleanse_data(tag):
    text=tag.get_text()
    pos,title,tv,date,members,rating,na,na = re.findall(r"\s*(.+)",text)
    match = re.search(r'(.+)Watch ((Episode)|(Promotional)) Video',title)
    if match:
        title = match.group(1)
    a=tag.find("a")
    return (int(pos), title, date.split('-')[0][:-1], date.split('-')[1][1:], float(rating), a.get('href'))

top_animes_url = 'https://myanimelist.net/topanime.php'
top_animes_soup = BeautifulSoup(requests.get(top_animes_url).text, 'html.parser')
next_limit = top_animes_soup.find("a", class_='next')
if next_limit:
    next_limit = next_limit.get('href')

tags = top_animes_soup.find_all("tr", class_='ranking-list')

entries = {'pos':[], 'title':[], 'start_date':[], 'end_date':[], 'rating':[], 'url': []}
for block in map(cleanse_data,tags):
    entries['pos'].append(block[0])
    entries['title'].append(block[1])
    entries['start_date'].append(block[2])
    entries['end_date'].append(block[3])
    entries['rating'].append(block[4])
    entries['url'].append(block[5])
while next_limit:
# for i in range(3):
    top_animes_soup = BeautifulSoup(requests.get(top_animes_url + next_limit ).text, 'html.parser')
    tags = top_animes_soup.find_all("tr", class_='ranking-list')
    for block in map(cleanse_data,tags):
        entries['pos'].append(block[0])
        entries['title'].append(block[1])
        entries['start_date'].append(block[2])
        entries['end_date'].append(block[3])
        entries['rating'].append(block[4])
        entries['url'].append(block[5])
    next_limit = top_animes_soup.find("a", class_='next')
    if next_limit:
        next_limit=next_limit.get('href')

anime_db = pd.DataFrame.from_dict(entries)

In [163]:
anime_db.shape

(2150, 6)

In [164]:
anime_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150 entries, 0 to 2149
Data columns (total 6 columns):
end_date      2150 non-null object
pos           2150 non-null int64
rating        2150 non-null float64
start_date    2150 non-null object
title         2150 non-null object
url           2150 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 100.9+ KB


In [139]:
anime_db.to_csv('anime_ranking')

In [140]:
url = 'https://myanimelist.net/api/anime/search.xml?q='
user='****'
password='****'

for query in anime_db.title[:3]:
    response = requests.get(url+query, auth=requests.auth.HTTPBasicAuth(user, password))
    soup = BeautifulSoup(response.text, 'html.parser')
    print(html.unescape(soup.synopsis.get_text()))

"In order for something to be obtained, something of equal value must be lost."<br />
<br />
Alchemy is bound by this Law of Equivalent Exchange—something the young brothers Edward and Alphonse Elric only realize after attempting human transmutation: the one forbidden act of alchemy. They pay a terrible price for their transgression—Edward loses his left leg, Alphonse his physical body. It is only by the desperate sacrifice of Edward's right arm that he is able to affix Alphonse's soul to a suit of armor. Devastated and alone, it is the hope that they would both eventually return to their original bodies that gives Edward the inspiration to obtain metal limbs called "automail" and become a state alchemist, the Fullmetal Alchemist.<br />
<br />
Three years of searching later, the brothers seek the Philosopher's Stone, a mythical relic that allows an alchemist to overcome the Law of Equivalent Exchange. Even with military allies Colonel Roy Mustang, Lieutenant Riza Hawkeye, and Lieutenan

In [ ]:
import sys
import time

def look_href(tag):
    return (tag.find_parent("td") and 
        tag.name == 'a' and
        tag.has_attr('href') and
        re.search(r'https://myanimelist.net/character/',tag.get('href')) and
        not tag.has_attr('class'))


character_entries = {'anime':[], 'char_name':[], 'char_url':[]} 
char_db = pd.DataFrame(columns=['anime','char_name','char_url'])
size = anime_db.shape[0]
percent_5 = size//5
count = 1
percents = 1
t0 = time.time()
for anime_link, title in anime_db[['url','title']].itertuples(index=False):
    if count % percent_5 == 0:
        sys.stdout.write(str(percents*5)+'%')
        sys.stdout.write(' -- '+str(time.time()-t0))
        t0 = time.time()
        percents=percents+1
    count = count + 1
    sys.stdout.write('.')
    anime_soup = BeautifulSoup(requests.get(anime_link).text, 'html.parser')
    characters_link = top_animes_soup.find("a",string="Characters & Staff").get('href')
    character_soup = BeautifulSoup(requests.get(characters_link).text, 'html.parser')
    tags = character_soup.find_all(look_href)
    char_db = char_db.append(pd.DataFrame.from_dict({'char_name': list(map(lambda x: x.get_text(), tags)),
                    'char_url': list(map(lambda x: x.get('href'), tags)),
                   'anime':[title for _ in range(len(tags))]}), ignore_index=True)


...................................................